In [12]:
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from func import ShopeeDataset, device, torch, np, f1_score_cal, display_df, path, path_img, get_data
import tensorflow as tf
import efficientnet.keras as efn
import math
from sklearn.neighbors import NearestNeighbors

In [2]:
# input
df = get_data(path / 'train.csv')
df.head()

,posting_id,image,image_phash,title,label_group,filepath,target,title_edit
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,/Users/kevin/OneDrive - Seagroup/computer_viso...,"[train_129225211, train_2278313361]",paper bag victoria secret
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,/Users/kevin/OneDrive - Seagroup/computer_viso...,"[train_3386243561, train_3423213080]","double tape 3m vhb 12 mm x 4,5 m original / do..."
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,/Users/kevin/OneDrive - Seagroup/computer_viso...,"[train_2288590299, train_3803689425]",maling tts canned pork luncheon meat 397 gr
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,/Users/kevin/OneDrive - Seagroup/computer_viso...,"[train_3342059966, train_2406599165]",daster batik lengan pendek - motif acak / camp...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,/Users/kevin/OneDrive - Seagroup/computer_viso...,"[train_3369186413, train_921438619]",nescafe \xc3\x89clair latte 220ml


In [3]:
image_paths = df.filepath.unique().tolist()

In [4]:
def get_model(s,m,bottleneck):

    ''' This function creates our model, we can choose from B0-B7 as our bottle neck model  '''

    margin = ArcMarginProduct(
        n_classes = N_CLASSES, 
        s = s, 
        m = m, 
        name='head/arc_margin', 
        dtype='float32'
        )

    inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
    label = tf.keras.layers.Input(shape = (), name = 'inp2')
    
    
    if bottleneck=='b0':
        x = efn.EfficientNetB0(weights = 'imagenet', include_top = False)(inp)           
    
    elif bottleneck=='b1':
        x = efn.EfficientNetB1(weights = 'imagenet', include_top = False)(inp)
        
    elif bottleneck=='b3':
        x = efn.EfficientNetB3(weights = 'imagenet', include_top = False)(inp)
        
    elif bottleneck=='b5':
        x = efn.EfficientNetB5(weights = 'imagenet', include_top = False)(inp)
    
    elif bottleneck=='b7':
        x = efn.EfficientNetB7(weights = 'imagenet', include_top = False)(inp)
        
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = margin([x, label])
    
    output = tf.keras.layers.Softmax(dtype='float32')(x)

    model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])

    opt = tf.keras.optimizers.Adam(learning_rate = LR)

    model.compile(
        optimizer = opt,
        loss = [tf.keras.losses.SparseCategoricalCrossentropy()],
        metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
        ) 
    
    return model

In [ ]:
def model_call(s,m,model_n,epochs):
    
    K.clear_session()
    s=s
    m=m
    model = get_model(s,m,model_n)

    # Model checkpoint
    checkpoint = tf.keras.callbacks.ModelCheckpoint(f'EfficientNet_{model_n}_{s}_{m}_{IMAGE_SIZE[0]}_{SEED}.h5', 
                                                monitor = 'val_loss', 
                                                verbose = VERBOSE, 
                                                save_best_only = True,
                                                save_weights_only = True, 
                                                mode = 'min')
    
    #Early Stopping
    early =tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    
    EPOCHS=epochs

    #Creating Model
    history = model.fit(train_dataset,
                    steps_per_epoch = STEPS_PER_EPOCH,
                    epochs = EPOCHS,
                    callbacks = [checkpoint, get_lr_callback(),early], 
                    validation_data = val_dataset,
                    verbose = VERBOSE)
    
    
    #Plotting the model accuracy, loss after compilation

    plt.plot(history.history['sparse_categorical_accuracy'])
    plt.plot(history.history['val_sparse_categorical_accuracy'])
    plt.title('Sparse Categorical Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epochs')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()


s=15
m=0.0001 
model='b3'
epochs=50
model = model_call(s,m,model,epochs)